# Predictions blender

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [3]:
import numpy as np
import pandas as pd

from data_utils import get_id_type_list_for_class, GENERATED_DATA, OUTPUT_PATH
from test_utils import create_submission

No test data found at D:\vfomin\MyExamples\python\planet_amazon_rainforest\input\test 
No additional test data found at D:\vfomin\MyExamples\python\planet_amazon_rainforest\input\test 


Using TensorFlow backend.


## Simple probability mean 

In [4]:
predictions_csv = glob(os.path.join(OUTPUT_PATH, "predictions_*.csv"))
predictions_csv

['D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_0_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_10_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_11_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_12_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_13_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_14_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_1_SqueezeNet21_all_classes_fold=4_seed=

In [5]:
predictions = []
for csv_filepath in predictions_csv:
    predictions.append(pd.read_csv(csv_filepath))

In [8]:
def compute_mean(predictions):
    df = predictions[0]
    for p in predictions[1:]:
        df = pd.concat([df, p], axis=0)
    df = df.apply(pd.to_numeric, errors='ignore')
    gb = df.groupby('image_name')
    df2 = gb.agg(np.mean).reset_index()
    return df2

In [9]:
mean_predictions_df = compute_mean(predictions)

In [10]:
mean_predictions_df.head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,file_0,0.075577,0.022042,0.028876,0.018089,0.032277,0.903471,0.013046,0.059221,0.093378,0.027963,0.020402,0.083580,0.967447,0.053680,0.035005,0.046575,0.088839
1,file_1,0.035373,0.011506,0.014149,0.007825,0.020209,0.938606,0.004963,0.037624,0.064372,0.013600,0.006733,0.056471,0.984725,0.029025,0.024612,0.022560,0.068315
2,file_10,0.449505,0.034339,0.059886,0.027366,0.057424,0.431175,0.063071,0.088553,0.169601,0.047239,0.087217,0.583021,0.913226,0.166780,0.118563,0.112523,0.232856
3,file_100,0.078833,0.020902,0.019649,0.012992,0.025159,0.939492,0.007644,0.047085,0.070833,0.030681,0.010401,0.059309,0.973639,0.079619,0.026858,0.032322,0.110833
4,file_1000,0.281819,0.043391,0.059090,0.035357,0.050177,0.832329,0.022377,0.087638,0.141859,0.083981,0.046980,0.133932,0.937326,0.180613,0.047837,0.055389,0.167381


In [11]:
thresholds = {
    'agriculture': 0.35,
    'artisinal_mine': 0.35,
    'bare_ground': 0.35,
    'blooming': 0.35,
    'blow_down': 0.1,
    'clear': 0.5,
    'cloudy': 0.35,
    'conventional_mine': 0.35,
    'cultivation': 0.35,
    'habitation': 0.35,
    'haze': 0.35,
    'partly_cloudy': 0.35,
    'primary': 0.5,
    'road': 0.35,
    'selective_logging': 0.35,
    'slash_burn': 0.1,
    'water': 0.35
}

In [14]:
create_submission(mean_predictions_df, info="squeezenet21_mean_5_folds", thresholds=thresholds)

## Train xgboost trees to make better predictions

In [ ]:
from data_utils import unique_tags, get_label

val_predictions_filepath = os.path.join(GENERATED_DATA, "val_predictions.csv")
if not os.path.exists(val_predictions_filepath):
    
    val_predictions_csv = glob(os.path.join(OUTPUT_PATH, "val_predictions_*.csv"))
    df = pd.read_csv(val_predictions_csv[0]).dropna()
    for filepath in val_predictions_csv[1:]:
        df = pd.concat([df, pd.read_csv(filepath).dropna()])
    df.reset_index(inplace=True)   
    df.drop('index', axis=1, inplace=True)
    df['image_id'] = df['image_name'].apply(lambda x: int(x[len('test_'):]))
    target_tags = ['target_' + t for t in unique_tags]
    for t in target_tags:
        df[t] = ''
    def fill_target_tags(row):
        image_id = row[0]
        labels = get_label(image_id, "Train_jpg")
        row[1:] = labels    
        return row
    
    df.to_csv(val_predictions_filepath, index=False)
    
else:
    df = pd.read_csv(val_predictions_filepath)

In [41]:
df.head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,test_628,0.071995,0.019810,0.018135,0.020460,0.015155,0.950393,0.018868,0.034112,0.073635,...,0,0,0,0,0,1,0,0,0,0
1,test_8842,0.070470,0.024685,0.020838,0.020732,0.016295,0.946937,0.013844,0.024798,0.065312,...,0,0,0,0,0,1,0,0,0,0
2,test_11877,0.760127,0.092261,0.118277,0.099680,0.101711,0.741155,0.107806,0.115136,0.207027,...,0,0,1,0,0,1,1,0,0,0
3,test_2413,0.229211,0.058638,0.096851,0.112940,0.040866,0.833260,0.051784,0.093998,0.156023,...,0,0,0,0,0,1,0,0,0,1
4,test_28838,0.071773,0.017712,0.019611,0.027369,0.016949,0.962932,0.013631,0.027329,0.066544,...,0,0,0,0,0,1,0,0,0,0


#### Check prediction of rare classes

In [42]:
from data_utils import equalized_data_classes, TRAIN_ENC_CL_CSV

In [43]:
image_ids = TRAIN_ENC_CL_CSV[TRAIN_ENC_CL_CSV['class_0'] == 1]['image_id']

In [47]:
rare_classes_predictions = df[df['image_id'].isin(image_ids)]

In [50]:
rare_classes_predictions.loc[17, list(tuple(unique_tags)) + list(tuple(target_tags))]

agriculture                  0.0397602
artisinal_mine               0.0172498
bare_ground                  0.0159017
blooming                     0.0199326
blow_down                    0.0100598
clear                         0.973735
cloudy                         0.00855
conventional_mine            0.0235309
cultivation                  0.0542153
habitation                  0.00319715
haze                         0.0224533
partly_cloudy                  0.10198
primary                        0.95575
road                         0.0193523
selective_logging            0.0158572
slash_burn                   0.0601998
water                         0.015458
target_agriculture                   0
target_artisinal_mine                0
target_bare_ground                   0
target_blooming                      1
target_blow_down                     0
target_clear                         1
target_cloudy                        0
target_conventional_mine             0
target_cultivation       

In [51]:
import xgboost as xgb

d:\vfomin\myexamples\venv3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### A simple try of xgb

In [52]:
from sklearn.model_selection import KFold

In [53]:
n_folds = 5

In [54]:
kf = KFold(n_splits=n_folds)

trainval_x = df[unique_tags].values
trainval_y = df[target_tags].values
    
for train_index, test_index in kf.split(trainval_x):
    train_x, val_x = trainval_x[train_index], trainval_x[test_index]
    train_y, val_y = trainval_y[train_index], trainval_y[test_index]
    
    print(train_x.shape, train_y.shape)
    print(train_x[:5, :], train_y[:5])
    print(val_x[:5, :], val_y[:5])
    break

(64512, 17) (64512, 17)
[[ 0.09093951  0.02054392  0.02418912  0.01544789  0.03247925  0.92379266
   0.00828369  0.04989937  0.10721629  0.0278988   0.00523409  0.08840037
   0.97719461  0.06283034  0.03732468  0.03562859  0.09488261]
 [ 0.05455842  0.01311656  0.02213255  0.01007489  0.03110504  0.91531706
   0.00602263  0.0469023   0.12840895  0.02287883  0.00794587  0.05563896
   0.98842609  0.03919864  0.03108688  0.02699875  0.06988449]
 [ 0.01405455  0.00637132  0.00833274  0.00247492  0.01473563  0.95992196
   0.00172729  0.03102464  0.03605482  0.00775872  0.00121341  0.03003966
   0.99548018  0.01660914  0.023029    0.01427789  0.05433382]
 [ 0.45359641  0.01025823  0.04004937  0.00729494  0.03303159  0.48314595
   0.02731058  0.06863047  0.13719328  0.02727759  0.02906365  0.58874065
   0.92141306  0.10537007  0.12347737  0.07908679  0.10379606]
 [ 0.1899071   0.01707581  0.02652191  0.01242715  0.06425259  0.72012264
   0.03139025  0.06736487  0.09192603  0.04307153  0.02066

In [61]:
params = {
    "objective": "binary:logistic",
    "booster": "gbtree",
    "eval_metric": "logloss",
    "learning_rate": 0.01,
    "tree_method": 'exact',
    "n_estimators": 100,
    "max_depth": 6,
#     "subsample": subsample,
#     "colsample_bytree": colsample_bytree,
    "silent": False,    
}
num_boost_round = 2000
early_stopping_rounds = 12

In [62]:
tag_index = 0

In [63]:
dtrain = xgb.DMatrix(train_x, train_y[:, tag_index])
dval = xgb.DMatrix(val_x, val_y[:, tag_index])

In [66]:
watchlist = [(dtrain, 'train'), (dval, 'eval')]

In [67]:
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

[0]	train-logloss:0.688044	eval-logloss:0.688636
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 12 rounds.
[1]	train-logloss:0.683043	eval-logloss:0.684329
[2]	train-logloss:0.678139	eval-logloss:0.679991
[3]	train-logloss:0.673332	eval-logloss:0.675851
[4]	train-logloss:0.668615	eval-logloss:0.671676
[5]	train-logloss:0.663989	eval-logloss:0.667557
[6]	train-logloss:0.659454	eval-logloss:0.66422
[7]	train-logloss:0.655002	eval-logloss:0.660262
[8]	train-logloss:0.650632	eval-logloss:0.656311
[9]	train-logloss:0.646342	eval-logloss:0.652608
[10]	train-logloss:0.642135	eval-logloss:0.648944
[11]	train-logloss:0.638003	eval-logloss:0.645748
[12]	train-logloss:0.633946	eval-logloss:0.642227
[13]	train-logloss:0.62996	eval-logloss:0.638848
[14]	train-logloss:0.626047	eval-logloss:0.635437
[15]	train-logloss:0.6222	eval-logloss:0.632199
[16]	train-logloss:0.61842	eval-logloss:0.628991
[17]	train-loglo

[161]	train-logloss:0.40162	eval-logloss:0.452414
[162]	train-logloss:0.401155	eval-logloss:0.452082
[163]	train-logloss:0.400696	eval-logloss:0.451761
[164]	train-logloss:0.400231	eval-logloss:0.451409
[165]	train-logloss:0.399788	eval-logloss:0.451197
[166]	train-logloss:0.399351	eval-logloss:0.45094
[167]	train-logloss:0.398899	eval-logloss:0.450497
[168]	train-logloss:0.398454	eval-logloss:0.450059
[169]	train-logloss:0.398015	eval-logloss:0.44963
[170]	train-logloss:0.397583	eval-logloss:0.449211
[171]	train-logloss:0.397148	eval-logloss:0.449026
[172]	train-logloss:0.396731	eval-logloss:0.448619
[173]	train-logloss:0.396312	eval-logloss:0.448366
[174]	train-logloss:0.395897	eval-logloss:0.448045
[175]	train-logloss:0.395493	eval-logloss:0.447797
[176]	train-logloss:0.395088	eval-logloss:0.44746
[177]	train-logloss:0.394693	eval-logloss:0.447122
[178]	train-logloss:0.394304	eval-logloss:0.446897
[179]	train-logloss:0.393919	eval-logloss:0.446538
[180]	train-logloss:0.393538	eval-l

In [80]:
y_preds_ = gbm.predict(xgb.DMatrix(trainval_x), ntree_limit=gbm.best_iteration+1)

In [81]:
y_preds = df[unique_tags].values
y_preds[:, tag_index] = y_preds_

In [82]:
y_true = df[target_tags].values

In [83]:
y_preds.shape, y_true.shape

((80640, 17), (80640, 17))

In [69]:
from metrics import score

In [88]:
score(y_true, y_preds > 0.35)

0.79164265811247403

In [87]:
score(y_true, df[unique_tags].values > 0.35)

0.78733284263456171

Run CV with a random search of optimal parameters

In [105]:
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
best_params_for_tag_index = {}
for tag_index in range(len(unique_tags)):
    
    print("\n----------------\n Tag index: {}".format(tag_index))
    iter_num = 50
    seed = 2017
    n_folds = 5
    dtrainval = xgb.DMatrix(trainval_x, label=trainval_y[:, tag_index], feature_names=unique_tags)

    best_params_for_tag_index[tag_index] = {
        'test_logloss_mean': 1e10,
        'params': None,
    }
    
    for z in range(iter_num):
        print("\n-- Iteration: {}".format(z))
        eta = np.random.uniform(0.005, 0.25)
        max_depth = np.random.randint(2, 6)
        subsample = np.random.uniform(0.5, 0.95)
        colsample_bytree = np.random.uniform(0.5, 0.95)
        seed += z-1
        print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth,subsample,colsample_bytree))
        params = {
            "objective": "binary:logistic",
            "booster": "gbtree",
            "eval_metric": "logloss",
            "eta": eta,
            "tree_method": 'exact',
            "max_depth": max_depth,
            "subsample": subsample,
            "colsample_bytree": colsample_bytree,
            "silent": 1,
            "seed": seed, 
        }
        num_boost_round = 2000
        early_stopping_rounds = 12

        cvresult = xgb.cv(params, dtrain=dtrainval,
                           seed=params['seed'], 
                           num_boost_round=num_boost_round, 
                           early_stopping_rounds=early_stopping_rounds, nfold=n_folds, verbose_eval=False)
                
        min_test_logloss_mean = cvresult['test-logloss-mean'].min()
        if best_params_for_tag_index[tag_index]['test_logloss_mean'] > min_test_logloss_mean:
            best_params_for_tag_index[tag_index]['test_logloss_mean'] = min_test_logloss_mean
            best_params_for_tag_index[tag_index]['params'] = params
            print("Best cv result: ", cvresult.loc[cvresult.index[-1], :])
            print("Best params: ", params)
            
        


----------------
 Tag index: 0
-- Iteration: 0

XGBoost params. ETA: 0.1449720818172992, MAX_DEPTH: 3, SUBSAMPLE: 0.8204967474962096, COLSAMPLE_BY_TREE: 0.7089159774987868
Best cv result:  test-logloss-mean     0.382224
test-logloss-std      0.003136
train-logloss-mean    0.360981
train-logloss-std     0.001019
Name: 243, dtype: float64
Best params:  {'objective': 'binary:logistic', 'booster': 'gbtree', 'eval_metric': 'logloss', 'eta': 0.1449720818172992, 'tree_method': 'exact', 'max_depth': 3, 'subsample': 0.8204967474962096, 'colsample_bytree': 0.7089159774987868, 'silent': 1, 'seed': 2016}
-- Iteration: 1

XGBoost params. ETA: 0.1449720818172992, MAX_DEPTH: 3, SUBSAMPLE: 0.8204967474962096, COLSAMPLE_BY_TREE: 0.7089159774987868
-- Iteration: 2

XGBoost params. ETA: 0.1449720818172992, MAX_DEPTH: 3, SUBSAMPLE: 0.8204967474962096, COLSAMPLE_BY_TREE: 0.7089159774987868
-- Iteration: 3

XGBoost params. ETA: 0.10836375674976262, MAX_DEPTH: 3, SUBSAMPLE: 0.6186701247268119, COLSAMPLE_BY_

-- Iteration: 46

XGBoost params. ETA: 0.010679805390037555, MAX_DEPTH: 2, SUBSAMPLE: 0.7712915472293789, COLSAMPLE_BY_TREE: 0.8472487349698614
-- Iteration: 47

XGBoost params. ETA: 0.09159703986540058, MAX_DEPTH: 3, SUBSAMPLE: 0.7834084121756955, COLSAMPLE_BY_TREE: 0.7214070723028566
-- Iteration: 48

XGBoost params. ETA: 0.010818978991139153, MAX_DEPTH: 2, SUBSAMPLE: 0.8293360878867488, COLSAMPLE_BY_TREE: 0.6748429921861105
-- Iteration: 49

XGBoost params. ETA: 0.09262393757042668, MAX_DEPTH: 4, SUBSAMPLE: 0.6565148379879179, COLSAMPLE_BY_TREE: 0.5748551275702991

----------------
 Tag index: 1
-- Iteration: 0

XGBoost params. ETA: 0.06060038588642189, MAX_DEPTH: 4, SUBSAMPLE: 0.8395564498918062, COLSAMPLE_BY_TREE: 0.5918176956145598
Best cv result:  test-logloss-mean     0.036059
test-logloss-std      0.001609
train-logloss-mean    0.029951
train-logloss-std     0.000481
Name: 166, dtype: float64
Best params:  {'objective': 'binary:logistic', 'booster': 'gbtree', 'eval_metric': 'l

-- Iteration: 41

XGBoost params. ETA: 0.22791487926078888, MAX_DEPTH: 5, SUBSAMPLE: 0.7908728611503619, COLSAMPLE_BY_TREE: 0.5374541192029424
-- Iteration: 42

XGBoost params. ETA: 0.13193724869098883, MAX_DEPTH: 2, SUBSAMPLE: 0.7022458730287486, COLSAMPLE_BY_TREE: 0.8776121141880973
-- Iteration: 43

XGBoost params. ETA: 0.20831237365660626, MAX_DEPTH: 3, SUBSAMPLE: 0.8688603088753073, COLSAMPLE_BY_TREE: 0.8673561381273192
-- Iteration: 44

XGBoost params. ETA: 0.22714269316782962, MAX_DEPTH: 2, SUBSAMPLE: 0.5355817962419311, COLSAMPLE_BY_TREE: 0.5528086371655933
-- Iteration: 45

XGBoost params. ETA: 0.21291598562857084, MAX_DEPTH: 4, SUBSAMPLE: 0.8367093737525602, COLSAMPLE_BY_TREE: 0.5174684221971676
-- Iteration: 46

XGBoost params. ETA: 0.010679805390037555, MAX_DEPTH: 2, SUBSAMPLE: 0.7712915472293789, COLSAMPLE_BY_TREE: 0.8472487349698614
-- Iteration: 47

XGBoost params. ETA: 0.09159703986540058, MAX_DEPTH: 3, SUBSAMPLE: 0.7834084121756955, COLSAMPLE_BY_TREE: 0.721407072302856

-- Iteration: 37

XGBoost params. ETA: 0.15526514252446666, MAX_DEPTH: 4, SUBSAMPLE: 0.7056974134393004, COLSAMPLE_BY_TREE: 0.6956641153604917
-- Iteration: 38

XGBoost params. ETA: 0.07107940829971568, MAX_DEPTH: 2, SUBSAMPLE: 0.765590577088374, COLSAMPLE_BY_TREE: 0.8064855363374669
-- Iteration: 39

XGBoost params. ETA: 0.10439059949422161, MAX_DEPTH: 5, SUBSAMPLE: 0.9240255018094838, COLSAMPLE_BY_TREE: 0.5104603847251689
-- Iteration: 40

XGBoost params. ETA: 0.2123537128679522, MAX_DEPTH: 5, SUBSAMPLE: 0.5364374719085715, COLSAMPLE_BY_TREE: 0.9248549177292057
-- Iteration: 41

XGBoost params. ETA: 0.22791487926078888, MAX_DEPTH: 5, SUBSAMPLE: 0.7908728611503619, COLSAMPLE_BY_TREE: 0.5374541192029424
-- Iteration: 42

XGBoost params. ETA: 0.13193724869098883, MAX_DEPTH: 2, SUBSAMPLE: 0.7022458730287486, COLSAMPLE_BY_TREE: 0.8776121141880973
-- Iteration: 43

XGBoost params. ETA: 0.20831237365660626, MAX_DEPTH: 3, SUBSAMPLE: 0.8688603088753073, COLSAMPLE_BY_TREE: 0.8673561381273192
-

-- Iteration: 39

XGBoost params. ETA: 0.10439059949422161, MAX_DEPTH: 5, SUBSAMPLE: 0.9240255018094838, COLSAMPLE_BY_TREE: 0.5104603847251689
-- Iteration: 40

XGBoost params. ETA: 0.2123537128679522, MAX_DEPTH: 5, SUBSAMPLE: 0.5364374719085715, COLSAMPLE_BY_TREE: 0.9248549177292057
-- Iteration: 41

XGBoost params. ETA: 0.22791487926078888, MAX_DEPTH: 5, SUBSAMPLE: 0.7908728611503619, COLSAMPLE_BY_TREE: 0.5374541192029424
-- Iteration: 42

XGBoost params. ETA: 0.13193724869098883, MAX_DEPTH: 2, SUBSAMPLE: 0.7022458730287486, COLSAMPLE_BY_TREE: 0.8776121141880973
-- Iteration: 43

XGBoost params. ETA: 0.20831237365660626, MAX_DEPTH: 3, SUBSAMPLE: 0.8688603088753073, COLSAMPLE_BY_TREE: 0.8673561381273192
-- Iteration: 44

XGBoost params. ETA: 0.22714269316782962, MAX_DEPTH: 2, SUBSAMPLE: 0.5355817962419311, COLSAMPLE_BY_TREE: 0.5528086371655933
-- Iteration: 45

XGBoost params. ETA: 0.21291598562857084, MAX_DEPTH: 4, SUBSAMPLE: 0.8367093737525602, COLSAMPLE_BY_TREE: 0.5174684221971676


-- Iteration: 38

XGBoost params. ETA: 0.07107940829971568, MAX_DEPTH: 2, SUBSAMPLE: 0.765590577088374, COLSAMPLE_BY_TREE: 0.8064855363374669
-- Iteration: 39

XGBoost params. ETA: 0.10439059949422161, MAX_DEPTH: 5, SUBSAMPLE: 0.9240255018094838, COLSAMPLE_BY_TREE: 0.5104603847251689
-- Iteration: 40

XGBoost params. ETA: 0.2123537128679522, MAX_DEPTH: 5, SUBSAMPLE: 0.5364374719085715, COLSAMPLE_BY_TREE: 0.9248549177292057
-- Iteration: 41

XGBoost params. ETA: 0.22791487926078888, MAX_DEPTH: 5, SUBSAMPLE: 0.7908728611503619, COLSAMPLE_BY_TREE: 0.5374541192029424
-- Iteration: 42

XGBoost params. ETA: 0.13193724869098883, MAX_DEPTH: 2, SUBSAMPLE: 0.7022458730287486, COLSAMPLE_BY_TREE: 0.8776121141880973
-- Iteration: 43

XGBoost params. ETA: 0.20831237365660626, MAX_DEPTH: 3, SUBSAMPLE: 0.8688603088753073, COLSAMPLE_BY_TREE: 0.8673561381273192
-- Iteration: 44

XGBoost params. ETA: 0.22714269316782962, MAX_DEPTH: 2, SUBSAMPLE: 0.5355817962419311, COLSAMPLE_BY_TREE: 0.5528086371655933
-

In [ ]:
best_params_for_tag_index

In [ ]:
import pickle 

xgb_best_params_filepath = os.path.join(GENERATED_DATA, 'xgb_best_params.pkl')

with open(xgb_best_params_filepath, 'wb') as handle:
    pickle.dump(best_params_for_tag_index, handle, protocol=pickle.HIGHEST_PROTOCOL)    

In [122]:
_best_params_for_tag_index = {
0: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.38209179999999998},
 1: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.036067200000000001},
 2: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.6334399849809398,
   'eta': 0.15055304521905347,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2019,
   'silent': 1,
   'subsample': 0.6186701247268119,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.086130599999999988},
 3: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5045164695165618,
   'eta': 0.1428101678052025,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.7215305289307261,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.041870600000000001},
 4: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.7089159774987868,
   'eta': 0.20210354378354375,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2017,
   'silent': 1,
   'subsample': 0.8204967474962096,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.015638200000000001},
 5: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.30505919999999997},
 6: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.068178600000000006},
 7: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.7089159774987868,
   'eta': 0.20210354378354375,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2017,
   'silent': 1,
   'subsample': 0.8204967474962096,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.013706000000000001},
 8: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.8579978891053311,
   'eta': 0.18574545610881862,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2044,
   'silent': 1,
   'subsample': 0.8611335020320647,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.29456899999999997},
 9: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5045164695165618,
   'eta': 0.1428101678052025,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.7215305289307261,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.20715659999999997},
 10: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.7089159774987868,
   'eta': 0.20210354378354375,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.8204967474962096,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.1200102},
 11: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.29197859999999998},
 12: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.1244864},
 13: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5045164695165618,
   'eta': 0.1428101678052025,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.7215305289307261,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.3228318},
 14: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.8398837534294123,
   'eta': 0.14522170765829945,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2022,
   'silent': 1,
   'subsample': 0.8734804475952236,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.041907600000000003},
 15: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.586741114680066,
   'eta': 0.08503519422605446,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2052,
   'silent': 1,
   'subsample': 0.9395104239451562,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.027125},
 16: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.38378999999999996}
}

{0: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.38209179999999998},
 1: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.036067200000000001},
 2: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.6334399849809398,
   'eta': 0.15055304521905347,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2019,
   'silent': 1,
   'subsample': 0.6186701247268119,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.0861305999999

Train 17 binary classifiers

In [ ]:
for tag_index in range(len(unique_tags)):
    
    print("\n----------------\n Tag index: {}".format(tag_index))
    dtrainval = xgb.DMatrix(trainval_x, label=trainval_y[:, tag_index], feature_names=unique_tags)
    
    params = best_params_for_tag_index[tag_index]['params']
    num_boost_round = 2500
    early_stopping_rounds = 12

    gbm = xgb.train(params, dtrain=dtrainval,
                    seed=params['seed'], 
                    num_boost_round=num_boost_round, 
                    early_stopping_rounds=early_stopping_rounds, 
                    verbose_eval=False)
                
    best_params_for_tag_index[tag_index]['gbm'] = gbm        

Boost test predictions

In [ ]:
predictions_csv = glob(os.path.join(OUTPUT_PATH, "predictions_*.csv"))

prediction_df = pd.read_csv(predictions_csv[0]).dropna()
for filepath in predictions_csv[1:]:
    prediction_df = pd.concat([prediction_df, pd.read_csv(filepath).dropna()])
prediction_df.reset_index(inplace=True)   
prediction_df.drop('index', axis=1, inplace=True)
